# Extraindo dados de poluição do ar da CETESB

Danilo Lessa Bernardineli (http://fap.if.usp.br/~danlessa)

## Dependências, definições e coisas globais

In [280]:
import requests as req
import numpy as np
from lxml import etree
import pandas as pd

## Exploração

Fuçando aleatóriamente pelo site da CETESB, encontrei os seguintes URLs que possibilitam a mineração de dados:

* https://servicos.cetesb.sp.gov.br/arcgis/rest/services/QUALAR/CETESB_QUALAR/MapServer/6/ - WebService que dá informações sobre as estações
* http://sistemasinter.cetesb.sp.gov.br/Ar/php/ar_dados_horarios_resultado.php - Site de consulta pública que permite obter dados de poluentes em qualquer dia e estação

O primeiro link fornece informações como ID, nome, endereço e coordenadas das estações, os quais podem ser usados como input no segundo URL para adquirir os dados dos poluentes

In [43]:
URI = "http://sistemasinter.cetesb.sp.gov.br/Ar/php/ar_dados_horarios_resultado.php"
data = "20/06/2018"
estacao = "65"
params = {"texData": data,
          "selEst": estacao}

In [267]:
r = req.post(URI, params)
raw = r.text
print(r.text[-300:])

spacing="0" cellpadding="0" align="center">
   <TR>
     <TD height="15" vAlign="TOP" align="middle">&nbsp;</TD>
   </TR>
   <TR>
     <TD height="20" vAlign="TOP" align="middle"><b class="font04">Dados sujeitos a valida&ccedil;&atilde;o posterior.<b></TD>
   </TR>
 </table>
 <p>&nbsp;</p>



In [268]:
html = etree.HTML(r.text)

In [269]:
xpath_0 = "//table[@class='font01']"
tables = html.xpath(xpath_0)

In [270]:
xpath_1 = ".//tr"
rows = tables[1].xpath(xpath_1)

In [271]:
xpath_2 = ".//td"
xpath_2t = ".//td/descendant::*/text()"
cols = rows[3].xpath(xpath_2)
cols[0].text

'02:00'

### Sistematizar tudo

In [272]:
xpath_0 = "//table[@class='font01']"
xpath_1 = ".//tr"
xpath_2 = ".//td"
xpath_2t = ".//td/descendant::*/text()"

output = []

tables = html.xpath(xpath_0)
for table in tables[1:]:
    table_output = []
    rows = table.xpath(xpath_1)
    
    for row in rows:
        row_output = []
        cols = row.xpath(xpath_2)    
        cols_2 = row.xpath(xpath_2t)
        for i in range(len(cols)):
            col = cols[i].text
            if col is None:
                if(len(cols_2) > 0):
                    col2 = cols_2[0]
                    row_output.append(col2)
            else:
                row_output.append(col)
        table_output.append(row_output)
    output.append(table_output)

### Interpretar

In [273]:
output_dict = {}

for table in output:
    medida = table[0][0]
    tipos = table[1]
    dados = np.array(table[2:]).T
    for i, tipo in enumerate(tipos):
        if len(tipos) > 1:
            coluna = medida + "_" + tipo
        else:
            coluna = medida
        output_dict[coluna] = dados[i]

In [281]:
pd.DataFrame(output_dict)

,Hora,MP10_Média,MP10_Índice /,TRS,O3_Média,O3_Índice /
0,01:00,33,26,3,24,10
1,02:00,35,28,4,18,7
2,03:00,37,29,2,16,6
3,04:00,39,31,2,14,6
4,05:00,39,31,2,11,4
5,06:00,41,32,2,9,4
6,07:00,41,32,2,8,3
7,08:00,41,32,2,9,4
8,09:00,41,32,4,9,4
9,10:00,41,32,12,11,4


## Extração

In [291]:
data = "19/06/2018"
estacao = "64"
xpath_0 = "//table[@class='font01']"
xpath_1 = ".//tr"
xpath_2 = ".//td"
xpath_2t = ".//td/descendant::*/text()"

# Extrair

params = {"texData": data,
          "selEst": estacao}
r = req.post(URI, params)
raw = r.text.replace("<br>", "").replace("<br/>", "").replace("e /", "e")
html = etree.HTML(raw)
          
# Processar
          
output = []

tables = html.xpath(xpath_0)
for table in tables[1:]:
    table_output = []
    rows = table.xpath(xpath_1)
    
    for row in rows:
        row_output = []
        cols = row.xpath(xpath_2)    
        cols_2 = row.xpath(xpath_2t)
        for i in range(len(cols)):
            col = cols[i].text
            if col is None:
                if(len(cols_2) > 0):
                    col2 = cols_2[0]
                    row_output.append(col2)
            else:
                row_output.append(col)
        table_output.append(row_output)
    output.append(table_output)

output_dict = {}
          
# Interpretar

for table in output:
    medida = table[0][0]
    tipos = table[1]
    dados = np.array(table[2:]).T
    for i, tipo in enumerate(tipos):
        if len(tipos) > 1:
            coluna = medida + "_" + tipo
        else:
            coluna = medida
        output_dict[coluna] = dados[i]
          
# Exportar
          
pd.DataFrame(output_dict)

,Hora,NO2_ Média,NO2_Índice,MP10_Média,MP10_Índice,O3_Média,O3_Índice
0,01:00,--,--,30,24,17,7
1,02:00,23,5,31,25,13,5
2,03:00,22,5,31,25,10,4
3,04:00,21,4,32,25,6,2
4,05:00,20,4,32,25,3,1
5,06:00,20,4,33,26,2,1
6,07:00,20,4,33,26,2,1
7,08:00,20,4,34,27,2,1
8,09:00,26,5,35,28,3,1
9,10:00,30,6,35,28,5,2
